In [1]:
pip install lightgbm optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [91]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/PS-S3/Ep7/train.csv'
file_key_2 = 'Tabular-Playground-Series/PS-S3/Ep7/test.csv'
file_key_3 = 'Tabular-Playground-Series/PS-S3/Ep7/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Fixing dates (https://www.kaggle.com/competitions/playground-series-s3e7/discussion/386655)
train['arrival_year_month'] = pd.to_datetime(train['arrival_year'].astype(str) + train['arrival_month'].astype(str), format = '%Y%m')
test['arrival_year_month'] = pd.to_datetime(test['arrival_year'].astype(str) + test['arrival_month'].astype(str), format = '%Y%m')

train.loc[train.arrival_date > train.arrival_year_month.dt.days_in_month, 'arrival_date'] = train.arrival_year_month.dt.days_in_month
test.loc[test.arrival_date > test.arrival_year_month.dt.days_in_month, 'arrival_date'] = test.arrival_year_month.dt.days_in_month

train.drop(columns = 'arrival_year_month', inplace = True)
test.drop(columns = 'arrival_year_month', inplace = True)

train['low_price_flag'] = np.where(train['avg_price_per_room'] < 30, 1, 0)
train['segment_0'] = np.where(train['market_segment_type'] == 0, 1, 0)
train['segment_1'] = np.where(train['market_segment_type'] == 1, 1, 0)
train['total_guests'] = train['no_of_adults'] + train['no_of_children']
train['stay_length'] = train['no_of_weekend_nights'] + train['no_of_week_nights']
train['stay_during_weekend'] = np.where(train['no_of_weekend_nights'] > 0, 1, 0)
train['quarter_1'] = np.where(train['arrival_month'] <= 3, 1, 0)
train['quarter_2'] = np.where(((train['arrival_month'] >= 4) & (train['arrival_month'] <= 6)), 1, 0)
train['quarter_3'] = np.where(((train['arrival_month'] >= 7) & (train['arrival_month'] <= 9)), 1, 0)
train['quarter_4'] = np.where(train['arrival_month'] >= 10, 1, 0)
train['segment_0_feature_1'] = np.where(((train['market_segment_type'] == 0) & (train['lead_time'] <= 90)), 1, 0)
train['segment_0_feature_2'] = np.where(((train['market_segment_type'] == 0) & (train['avg_price_per_room'] > 98)), 1, 0)
train['segment_1_feature_1'] = np.where(((train['market_segment_type'] == 1) & (train['no_of_special_requests'] == 0)), 1, 0)
train['segment_1_feature_2'] = np.where(((train['market_segment_type'] == 1) & (train['no_of_special_requests'] > 0) & (train['lead_time'] <= 150)), 1, 0)
train['segment_0_year_flag'] = np.where(((train['market_segment_type'] == 0) & (train['arrival_year'] == 2018)), 1, 0)
train['segment_1_year_flag'] = np.where(((train['market_segment_type'] == 1) & (train['arrival_year'] == 2018)), 1, 0)
train['price_lead_time_flag'] = np.where(((train['avg_price_per_room'] > 100) & (train['lead_time'] > 150)), 1, 0)

test['low_price_flag'] = np.where(test['avg_price_per_room'] < 30, 1, 0)
test['segment_0'] = np.where(test['market_segment_type'] == 0, 1, 0)
test['segment_1'] = np.where(test['market_segment_type'] == 1, 1, 0)
test['total_guests'] = test['no_of_adults'] + test['no_of_children']
test['stay_length'] = test['no_of_weekend_nights'] + test['no_of_week_nights']
test['stay_during_weekend'] = np.where(test['no_of_weekend_nights'] > 0, 1, 0)
test['quarter_1'] = np.where(test['arrival_month'] <= 3, 1, 0)
test['quarter_2'] = np.where(((test['arrival_month'] >= 4) & (test['arrival_month'] <= 6)), 1, 0)
test['quarter_3'] = np.where(((test['arrival_month'] >= 7) & (test['arrival_month'] <= 9)), 1, 0)
test['quarter_4'] = np.where(test['arrival_month'] >= 10, 1, 0)
test['segment_0_feature_1'] = np.where(((test['market_segment_type'] == 0) & (test['lead_time'] <= 90)), 1, 0)
test['segment_0_feature_2'] = np.where(((test['market_segment_type'] == 0) & (test['avg_price_per_room'] > 98)), 1, 0)
test['segment_1_feature_1'] = np.where(((test['market_segment_type'] == 1) & (test['no_of_special_requests'] == 0)), 1, 0)
test['segment_1_feature_2'] = np.where(((test['market_segment_type'] == 1) & (test['no_of_special_requests'] > 0) & (test['lead_time'] <= 150)), 1, 0)
test['segment_0_year_flag'] = np.where(((test['market_segment_type'] == 0) & (test['arrival_year'] == 2018)), 1, 0)
test['segment_1_year_flag'] = np.where(((test['market_segment_type'] == 1) & (test['arrival_year'] == 2018)), 1, 0)
test['price_lead_time_flag'] = np.where(((test['avg_price_per_room'] > 100) & (test['lead_time'] > 150)), 1, 0)

# Splitting Duplicates

In [32]:
train_dup = train.copy()
test_dup = test.copy()

duplicates = pd.merge(train, test, on = train_dup.columns.tolist()[1:18])
train_dup_ids = duplicates['id_x'].tolist()
test_dup_ids = duplicates['id_y'].tolist()

## Removing duplicates
train_clean = train[~np.isin(train['id'], train_dup_ids)].reset_index(drop = True)
train_dup = train[np.isin(train['id'], train_dup_ids)].reset_index(drop = True)

test_clean = test[~np.isin(test['id'], test_dup_ids)].reset_index(drop = True)
test_dup = test[np.isin(test['id'], test_dup_ids)].reset_index(drop = True)

In [38]:
train_clean.head()

,id,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status,low_price_flag,segment_0,segment_1,total_guests,stay_length,stay_during_weekend,quarter_1,quarter_2,quarter_3,quarter_4,segment_0_feature_1,segment_0_feature_2,segment_1_feature_1,segment_1_feature_2,segment_0_year_flag,segment_1_year_flag,price_lead_time_flag
0,0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0,0,0,1,2,2,0,1,0,0,0,0,0,1,0,0,1,0
1,1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0,0,1,0,2,3,1,0,0,1,0,0,0,0,0,1,0,0
2,2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0,0,1,0,2,1,0,0,0,0,1,0,0,0,0,1,0,0
3,3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0,0,0,1,1,2,0,0,0,0,1,0,0,1,0,0,1,0
4,4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1,0,1,0,2,1,1,0,0,0,1,0,1,0,0,1,0,0


# Feature Selection

In [90]:
X = train_clean.drop(columns = ['id', 'low_price_flag', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'booking_status'], axis = 1)
Y = train_clean['booking_status']

## Running RFECV multiple times
RFE_results = list()

for i in tqdm(range(0, 10)):
    
    auto_feature_selection = RFECV(estimator = LGBMClassifier(), step = 1, min_features_to_select = 2, cv = 5, scoring = 'roc_auc').fit(X, Y)
    
    ## Extracting and storing features to be selected
    RFE_results.append(auto_feature_selection.support_)

## Changing to data-frame
RFE_results = pd.DataFrame(RFE_results)
RFE_results.columns = X.columns

## Computing the percentage of time features are flagged as important
RFE_results = 100*RFE_results.apply(np.sum, axis = 0) / RFE_results.shape[0]

## Identifying features with a percentage score > 80%
features_to_select = RFE_results.index[RFE_results > 80].tolist()
features_to_select

  0%|          | 0/10 [00:16<?, ?it/s]


KeyboardInterrupt: 

# Optuna Optimization

In [36]:
X = train_clean[features_to_select]
Y = train_clean['booking_status']

test_lgb = test_clean[features_to_select]

class Objective:

    def __init__(self, seed):
        # Hold this implementation specific arguments as the fields of the class.
        self.seed = seed

    def __call__(self, trial):
        
        ## Parameters to be evaluated
        param = dict(objective = 'binary',
                     metric = 'auc',
                     tree_method = 'gbdt', 
                     n_estimators = trial.suggest_int('n_estimators', 300, 10000),
                     learning_rate = trial.suggest_float('learning_rate', 0.001, 1, log = True),
                     max_depth = trial.suggest_int('max_depth', 3, 12),
                     lambda_l1 = trial.suggest_float('lambda_l1', 0.01, 10.0, log = True),
                     lambda_l2 = trial.suggest_float('lambda_l2', 0.01, 10.0, log = True),
                     num_leaves = trial.suggest_int('num_leaves', 2, 100),
                     bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9),
                     feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9)
#                      device = 'gpu'
                    )

        scores = []
        
        skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = self.seed)

        for train_idx, valid_idx in skf.split(X, Y):

            X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            Y_train , Y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

            model = LGBMClassifier(**param).fit(X_train, Y_train)

            preds_valid = model.predict_proba(X_valid)[:, 1]

            score = roc_auc_score(Y_valid, preds_valid)
            scores.append(score)

        return np.mean(scores)
    
## Defining SEED and Trials
SEED = 42
N_TRIALS = 5

# Execute an optimization
study = optuna.create_study(direction = 'maximize')
study.optimize(Objective(SEED), n_trials = N_TRIALS)

[I 2023-02-23 21:43:22,213] A new study created in memory with name: no-name-e4578b50-bf4e-4253-8fe2-f8b0b0090f6d


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.1860791647142356, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1860791647142356
[LightGBM] [Warning] feature_fraction is set=0.25789335238346806, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25789335238346806
[LightGBM] [Warning] lambda_l2 is set=0.013472187232704954, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.013472187232704954
[LightGBM] [Warning] bagging_fraction is set=0.344702766711013, subsample=1.0 will be ignored. Current value: bagging_fraction=0.344702766711013
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.1860791647142356, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1860791647142356
[LightGBM] [Warning] feature_fraction is set=0.25789335238346806, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25789335238346806
[LightGBM] [Warning] lambda_l2 is s

[I 2023-02-23 21:43:29,680] Trial 0 finished with value: 0.8925918835161163 and parameters: {'n_estimators': 689, 'learning_rate': 0.009088430830814133, 'max_depth': 6, 'lambda_l1': 0.1860791647142356, 'lambda_l2': 0.013472187232704954, 'num_leaves': 72, 'bagging_fraction': 0.344702766711013, 'feature_fraction': 0.25789335238346806}. Best is trial 0 with value: 0.8925918835161163.


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.09755223772612016, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.09755223772612016
[LightGBM] [Warning] feature_fraction is set=0.6935430480699973, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6935430480699973
[LightGBM] [Warning] lambda_l2 is set=5.522276495642769, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.522276495642769
[LightGBM] [Warning] bagging_fraction is set=0.7918406607989086, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7918406607989086
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.09755223772612016, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.09755223772612016
[LightGBM] [Warning] feature_fraction is set=0.6935430480699973, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6935430480699973
[LightGBM] [Warning] lambda_l2 is set=5

[I 2023-02-23 21:43:59,860] Trial 1 finished with value: 0.896099248077336 and parameters: {'n_estimators': 5196, 'learning_rate': 0.10153364370307941, 'max_depth': 9, 'lambda_l1': 0.09755223772612016, 'lambda_l2': 5.522276495642769, 'num_leaves': 18, 'bagging_fraction': 0.7918406607989086, 'feature_fraction': 0.6935430480699973}. Best is trial 1 with value: 0.896099248077336.


...

[I 2023-02-23 21:45:23,963] Trial 2 finished with value: 0.898467364765812 and parameters: {'n_estimators': 6496, 'learning_rate': 0.03048501353767569, 'max_depth': 8, 'lambda_l1': 0.08671455064873423, 'lambda_l2': 4.278383759668128, 'num_leaves': 49, 'bagging_fraction': 0.6084945955595327, 'feature_fraction': 0.491999828980985}. Best is trial 2 with value: 0.898467364765812.


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.1881764384240038, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1881764384240038
[LightGBM] [Warning] feature_fraction is set=0.37683273365528885, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.37683273365528885
[LightGBM] [Warning] lambda_l2 is set=2.995937246344799, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.995937246344799
[LightGBM] [Warning] bagging_fraction is set=0.3436058779527982, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3436058779527982
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.1881764384240038, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1881764384240038
[LightGBM] [Warning] feature_fraction is set=0.37683273365528885, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.37683273365528885
[LightGBM] [Warning] lambda_l2 is set=2

[I 2023-02-23 21:45:53,316] Trial 3 finished with value: 0.895924433559823 and parameters: {'n_estimators': 3619, 'learning_rate': 0.13875352348168177, 'max_depth': 6, 'lambda_l1': 0.1881764384240038, 'lambda_l2': 2.995937246344799, 'num_leaves': 22, 'bagging_fraction': 0.3436058779527982, 'feature_fraction': 0.37683273365528885}. Best is trial 2 with value: 0.898467364765812.


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.036318906893076194, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.036318906893076194
[LightGBM] [Warning] feature_fraction is set=0.868507389588629, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.868507389588629
[LightGBM] [Warning] lambda_l2 is set=0.059822034461512026, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.059822034461512026
[LightGBM] [Warning] bagging_fraction is set=0.7708777453025581, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7708777453025581
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.036318906893076194, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.036318906893076194
[LightGBM] [Warning] feature_fraction is set=0.868507389588629, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.868507389588629
[LightGBM] [Warning] lambda_l2 is

[I 2023-02-23 21:46:25,277] Trial 4 finished with value: 0.8897908089959181 and parameters: {'n_estimators': 3704, 'learning_rate': 0.07644323171900136, 'max_depth': 7, 'lambda_l1': 0.036318906893076194, 'lambda_l2': 0.059822034461512026, 'num_leaves': 35, 'bagging_fraction': 0.7708777453025581, 'feature_fraction': 0.868507389588629}. Best is trial 2 with value: 0.898467364765812.


# CV Procedure

In [37]:
cv_scores, roc_auc_scores = list(), list()
preds = list() 

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
        ## Building RF model
        lgb_md = LGBMClassifier(**study.best_trial.params).fit(X_train, Y_train)
        
        ## Predicting on X_test and test
        lgb_pred_1 = lgb_md.predict_proba(X_test)[:, 1]
        lgb_pred_2 = lgb_md.predict_proba(test_lgb)[:, 1]
        
        ## Computing roc-auc score
        roc_auc_scores.append(roc_auc_score(Y_test, lgb_pred_1))
        preds.append(lgb_pred_2)
        
    cv_scores.append(np.mean(roc_auc_scores))

[LightGBM] [Warning] lambda_l1 is set=0.08671455064873423, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08671455064873423
[LightGBM] [Warning] feature_fraction is set=0.491999828980985, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.491999828980985
[LightGBM] [Warning] lambda_l2 is set=4.278383759668128, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.278383759668128
[LightGBM] [Warning] bagging_fraction is set=0.6084945955595327, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6084945955595327
[LightGBM] [Warning] lambda_l1 is set=0.08671455064873423, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08671455064873423
[LightGBM] [Warning] feature_fraction is set=0.491999828980985, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.491999828980985
[LightGBM] [Warning] lambda_l2 is set=4.278383759668128, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.278383759668128
[LightGBM] [War

In [39]:
lgb_cv_score = np.mean(cv_scores)    
print('The oof roc-auc score over 5-folds (run 5 times) is:', lgb_cv_score)

The oof roc-auc score over 5-folds (run 5 times) is: 0.8984673647658121


In [92]:
lgb_preds_test = pd.DataFrame(preds).apply(np.mean, axis = 0)
clean_pred = pd.DataFrame({'id': test_clean['id']})
clean_pred['booking_status_clean'] = lgb_preds_test
clean_pred.head()

,id,booking_status_clean
0,42100,0.021243
1,42101,0.037615
2,42102,0.159126
3,42103,0.011049
4,42104,0.741003


In [93]:
dup_pred = duplicates[['id_y', 'booking_status']]
dup_pred.columns = ['id', 'booking_status_dup']
dup_pred['booking_status_dup'] = 1 - dup_pred['booking_status_dup']
dup_pred

/tmp/ipykernel_10175/3624402071.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup_pred['booking_status_dup'] = 1 - dup_pred['booking_status_dup']


,id,booking_status_dup
0,57409,1
1,51069,1
2,65654,0
3,53459,0
4,45279,0
...,...,...
716,43885,0
717,49594,0
718,45238,1
719,53480,1


In [94]:
# submission = pd.read_csv(file_content_stream_3)
submission.head()

,id,booking_status
0,42100,0.392
1,42101,0.392
2,42102,0.392
3,42103,0.392
4,42104,0.392


In [95]:
submission = pd.merge(submission.drop(columns = 'booking_status', axis = 1), clean_pred, on = 'id', how = 'left')
submission = pd.merge(submission, dup_pred, on = 'id', how = 'left')
submission['booking_status'] = np.where(np.isnan(submission['booking_status_clean']), submission['booking_status_dup'], submission['booking_status_clean'])
submission.drop(columns = ['booking_status_clean', 'booking_status_dup'], axis = 1, inplace = True)
submission.head()

,id,booking_status
0,42100,0.021243
1,42101,0.037615
2,42102,0.159126
3,42103,0.011049
4,42104,0.741003
